In [1]:
import os
import sys
import time
import glob
import numpy as np
import logging
import argparse
import tensorflow as tf
import numpy as np
import utils
from tqdm import tqdm
import shutil
from model_search import Network
from architect_graph import Architect

import time
from genotypes import PRIMITIVES
from genotypes import Genotype
from scipy.special import softmax

In [2]:
# tf.enable_eager_execution()
tf.set_random_seed(6969)
np.random.seed(6969)

In [3]:
import utils
from genotypes import Genotype

In [4]:
args = {
    "momentum": 0.9,
    "weight_decay": 3e-4,
    "arch_learning_rate": 3e-1,
    "momentum": 0.9,
    "grad_clip": 5,
    "learning_rate": 0.9,
    "learning_rate_decay": 0.97,
    "learning_rate_min": 0.0001,
    "num_batches_per_epoch": 2000,
    
    "unrolled": True,
    "epochs": 10,
    "train_batch_size": 8,
    "eval_batch_size": 8,
    "save": "EXP",
    "init_channels": 3,
    "num_layers": 3,
    "num_classes": 6,
    "crop_size": [8, 8],
    "save_checkpoints_steps": 100,
    "model_dir": 'gs://unet-darts/train-search-ckptss',
    "max_steps": 10000,
    # NEW
    "steps_per_eval": 2,
    "num_train_examples": 16,
    "num_batches_per_epoch": 2,
    #
    "seed": 6969,
    
    "use_tpu": False,
    "use_host_call": False,
    "tpu": 'unet-darts',
#     "zone": 'us-central1-f',
#     "project": "isro-nas"
    "zone": None,
    "project": None
}

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

args = Struct(**args)

### input_fn

In [5]:
def make_inp_fn(filename, mode, batch_size):
    
    def _input_fn(params):
        image_dataset = tf.data.TFRecordDataset(filename)
        W, H = args.crop_size[0], args.crop_size[1]

        # Create a dictionary describing the features.  
        image_feature_description = {
            'train_name': tf.FixedLenFeature([], tf.string),  
            'train_x': tf.FixedLenFeature([], tf.string),
            'train_y': tf.FixedLenFeature([], tf.string),
            'valid_name': tf.FixedLenFeature([], tf.string),  
            'valid_x': tf.FixedLenFeature([], tf.string),
            'valid_y': tf.FixedLenFeature([], tf.string)
        }
        def _parse_image_function(example_proto):
            # Parse the input tf.Example proto using the dictionary above.
            feature= tf.parse_single_example(example_proto, image_feature_description)
            train_x = feature['train_x']
            train_y = feature['train_y']
            valid_x = feature['valid_x']
            valid_y = feature['valid_y']
            train_name = feature['train_name']
            valid_name = feature['valid_name']

            train_x = tf.image.decode_png(train_x, channels=3)
            train_y = tf.image.decode_png(train_y, channels=3)
            valid_x = tf.image.decode_png(valid_x, channels=3)
            valid_y = tf.image.decode_png(valid_y, channels=3)
            
            
            train_x = tf.cast(train_x, tf.float32)
            train_x = tf.image.resize(train_x, (W, H))
            train_y = tf.cast(train_y, tf.float32)
            train_y = tf.image.resize(train_y, (W, H))
            valid_x = tf.cast(valid_x, tf.float32)
            valid_x = tf.image.resize(valid_x, (W, H))
            valid_y = tf.cast(valid_y, tf.float32)
            valid_y = tf.image.resize(valid_y, (W, H))
            
            train_y = train_y[:, :, 0]
            train_y = tf.expand_dims(train_y, axis=-1)
            
            valid_y = valid_y[:, :, 0]
            valid_y = tf.expand_dims(valid_y, axis=-1)
            
            return ((train_x, valid_x), (train_y, valid_y))

        dataset = image_dataset.map(_parse_image_function)
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            num_epochs = 1 # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size,  drop_remainder=True)

        return dataset
    
    return _input_fn

In [6]:
def make_inp_fn2(filename, mode, batch_size):
    
    def _input_fn(params):
        W, H = args.crop_size[0], args.crop_size[1]
        NUM_IMAGES = 20
        x_train = np.random.randint(0, 256, (NUM_IMAGES, W, H, 3)).astype(np.float32)
        y_train = np.random.randint(0, args.num_classes, (NUM_IMAGES, W, H, 1)).astype(np.float32)
        x_valid = np.random.randint(0, 256, (NUM_IMAGES, W, H, 3)).astype(np.float32)
        y_valid = np.random.randint(0, args.num_classes, (NUM_IMAGES, W, H, 1)).astype(np.float32)

        ds = (x_train, x_valid), (y_train, y_valid)
        dataset = tf.data.Dataset.from_tensor_slices(ds)
        num_epochs = None # indefinitely
        dataset = dataset.shuffle(buffer_size = 10 * batch_size)

        dataset = dataset.repeat(num_epochs).batch(batch_size, drop_remainder=True)
        return dataset
    
    return _input_fn

### Model function

In [7]:
class GeneSaver(tf.estimator.SessionRunHook):
    def __init__(self, model, alphas_normal, alphas_reduce):
        self.model = model
        self.alphas_normal = alphas_normal
        self.alphas_reduce = alphas_reduce
    
    def begin(self):
        self.global_step = tf.train.get_or_create_global_step()
        
    def end(self, session):
        alphas_normal, alphas_reduce = session.run([self.alphas_normal, self.alphas_reduce])
        alphas_normal = softmax(alphas_normal)
        alphas_reduce = softmax(alphas_reduce)
        
        genotype = self.model.genotype(alphas_normal, alphas_reduce)        
        self.global_step = session.run(self.global_step)
        
        filename = 'final_genotype.{}'.format((self.global_step))
        tf.logging.info("Saving Genotype for step: {}".format(str(self.global_step)))
        utils.write_genotype(genotype, filename)

In [8]:
def model_fn(features, labels, mode, params):
    criterion = tf.losses.softmax_cross_entropy
    model = Network(C=args.init_channels, net_layers=args.num_layers, criterion=criterion, num_classes=args.num_classes)
#     _ = model(np.random.randn(args.train_batch_size, args.crop_size[0], args.crop_size[1], 3))
    eval_hooks = None
    loss = None
    train_op = None
    eval_metric_ops = None
    prediction_dict = None
    export_outputs = None
    host_call = None
    
    # 2. Loss function, training/eval ops
    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step = tf.train.get_global_step()
        learning_rate_min = tf.constant(args.learning_rate_min)

        learning_rate = tf.train.exponential_decay(
            args.learning_rate,
            global_step,
            decay_rate=args.learning_rate_decay,
            decay_steps=args.num_batches_per_epoch,
            staircase=True,
        )

        lr = tf.maximum(learning_rate, learning_rate_min)

        optimizer = tf.train.MomentumOptimizer(lr, args.momentum)

        if(args.use_tpu):
            optimizer = tf.tpu.CrossShardOptimizer(optimizer)
        (x_train, x_valid) = features
        (y_train, y_valid) = labels

        preds = model(x_train)
        architect = Architect(model, args)
       # architect step
        
        update_ops0 = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops0):
            architect_step = architect.step(input_train=x_train,
                                            target_train=y_train,
                                            input_valid=x_valid,
                                            target_valid=y_valid,
                                            unrolled=args.unrolled,
                                            )
        w_var = model.get_thetas()
        loss = model._loss(preds, y_train)
        grads = tf.gradients(loss, w_var, name='train_gradients')
        
        with tf.control_dependencies([architect_step]):
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                train_op = optimizer.apply_gradients(zip(grads, w_var), global_step=tf.train.get_global_step())


    # 5. Return EstimatorSpec
    return tf.estimator.tpu.TPUEstimatorSpec(
        mode = mode,
        predictions = prediction_dict,
        loss = loss,
        train_op = train_op,
        eval_metrics = eval_metric_ops,
        export_outputs = export_outputs,
        evaluation_hooks=eval_hooks,
        host_call=host_call
    )

### Estimator

In [9]:
# Create functions to read in respective datasets
def get_train():
    return make_inp_fn2(filename = tf.gfile.Glob('../../datasets/train_search/train-search-*-00008.tfrecords'),
                        mode = tf.estimator.ModeKeys.TRAIN,
                        batch_size = args.train_batch_size)

In [10]:
# Create serving input function
def serving_input_fn():
    feature_placeholders = {
      IMAGE_LOC: tf.placeholder(tf.float32, [None])
    }
    
    feature_placeholders['IMAGES'] = tf.placeholder(tf.float32, [None, args.crop_size[0], args.crop_size[1], args.init_channels])
    
    features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
    }

    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

In [11]:
tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
      tpu=args.tpu,
      zone=args.zone,
      project=args.project)

In [12]:
config = tf.estimator.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=args.model_dir,
    save_checkpoints_steps=args.save_checkpoints_steps,
    tpu_config=tf.contrib.tpu.TPUConfig(num_shards=8),
    tf_random_seed=args.seed
)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [13]:
estimator = tf.estimator.tpu.TPUEstimator(
    use_tpu=args.use_tpu,
    model_fn=model_fn,
    config=config,
    train_batch_size=args.train_batch_size,
    eval_batch_size=args.eval_batch_size,
    params=vars(args)
)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.240.1.18:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2bb1a96e50>, '_model_dir': 'gs://unet-darts/train-search-ckptss', '_protocol': None, '_save_checkpoints_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tpu_config': TPUConfig(iterations_per_loop=2, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=2, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None, eval_training_input_configuration=2, experimental_host_call_every_n_steps=1), '_tf_random_seed': 6969, '_save_summary_steps': 100, '_device_fn': None, '_session_creation_time

In [14]:
estimator.train(input_fn = get_train(), steps=10)
# Loss for final step: 1.9186516.

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running train on CPU









INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from gs://unet-darts/train-search-ckptss/model.ckpt-290
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 290 into gs://unet-darts/train-search-ckptss/model.ckpt.
INFO:tensorflow:Saving checkpoints for 300 into gs://unet-darts/train-search-ckptss/model.ckpt.
Instructions for updating:
Use stand

In [15]:
estimator.get_variable_value('network/alphas_normal')
# array([[ 0.01695889, -0.04184179, -0.01425209,  0.04107325],

array([[ 0.01695889, -0.04184179, -0.01425209,  0.04107325],
       [-0.04151864,  0.00794181,  0.08955112, -0.05951698],
       [ 0.02892872, -0.04694319, -0.01564714,  0.03687111],
       [-0.00940474, -0.00713697,  0.04399676, -0.02644248],
       [ 0.00102057, -0.03073446, -0.02404075,  0.05464585],
       [ 0.02468788,  0.0012726 , -0.045113  ,  0.0225525 ],
       [ 0.01003213, -0.00243887, -0.05038981,  0.04622243],
       [ 0.01834823, -0.01843441, -0.02245464,  0.02499198],
       [ 0.02915651, -0.02340005, -0.02678607,  0.02285247],
       [ 0.01329712, -0.00685036, -0.02164326,  0.01748462],
       [-0.02285887,  0.0109119 ,  0.02861176, -0.01453894],
       [-0.00064974,  0.00058517,  0.00407862, -0.00187617],
       [-0.0025976 ,  0.00103705,  0.00718386, -0.00399489],
       [-0.00473083,  0.00820372,  0.01349145, -0.01548955]],
      dtype=float32)

In [16]:
estimator.get_variable_value('network/sequential/conv2d/kernel:0')

array([[[[ 0.21160507, -0.21848902,  0.23653212,  0.11754844,
           0.07842084,  0.20762058, -0.02427896,  0.05266505,
           0.06588185],
         [-0.10964472, -0.19849841,  0.20948522, -0.08818717,
          -0.1918711 ,  0.14931907, -0.10729859, -0.17201567,
          -0.11432522],
         [-0.12347911,  0.16643406, -0.05902734, -0.03249671,
           0.11748508,  0.14278087, -0.05715536,  0.21601404,
          -0.22197345]],

        [[-0.19773257,  0.21238694, -0.08473241,  0.08368283,
          -0.11171204, -0.11749534, -0.12242602, -0.14338504,
          -0.21754499],
         [ 0.11206506, -0.10706716,  0.08288258,  0.09993035,
           0.04749881,  0.07117351,  0.05172399, -0.18064596,
          -0.08402789],
         [-0.11511762,  0.21876034,  0.06738853,  0.0756783 ,
           0.12194356, -0.16608292,  0.0934642 ,  0.01872056,
          -0.22174531]],

        [[ 0.22120355, -0.08398765, -0.07401478,  0.13258912,
          -0.20800927,  0.12645443, -0.0617769

## Extra Test

In [14]:
criterion = tf.losses.softmax_cross_entropy
model = Network(C=args.init_channels, net_layers=args.num_layers, criterion=criterion, num_classes=args.num_classes)

In [15]:
(x_train, x_valid), (y_train, y_valid) = get_train()(vars(args)).make_one_shot_iterator().get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [16]:
logits = model(x_train)

In [14]:
loss = model._loss(logits, y_train)

(<tf.Tensor 'one_hot:0' shape=(8, 4, 4, 6) dtype=float32>, <tf.Tensor 'network/softmaxConv/softmax/Softmax:0' shape=(8, 4, 4, 6) dtype=float32>)


In [16]:
optimizer = tf.train.GradientDescentOptimizer(0.001)
optimizer.minimize(loss, var_list=model.get_thetas())

<tf.Operation 'GradientDescent' type=NoOp>

In [17]:
y_train

<tf.Tensor 'IteratorGetNext:2' shape=(8, 4, 4, 1) dtype=float32>

In [18]:
logits

<tf.Tensor 'network/softmaxConv/softmax/Softmax:0' shape=(8, 4, 4, 6) dtype=float32>

In [20]:
b, w, h, c = y_train.shape
y = tf.reshape(tf.cast(y_train, tf.int64), (b, w, h))
y = tf.one_hot(y, args.num_classes, on_value=1.0, off_value=0.0)

In [21]:
y

<tf.Tensor 'one_hot:0' shape=(8, 4, 4, 6) dtype=float32>

In [53]:
miou = tf.metrics.mean_iou(
    labels=y,
    predictions=tf.round(logits),
    num_classes=args.num_classes
)

In [50]:
b = tf.metrics.mean_iou(
    labels=tf.constant([[1, 0, 0], [0, 1, 0]]),
    predictions=tf.constant([[1, 0, 0], [0, 0.99999, 0]]),
    num_classes=2
)

In [54]:
local_op = tf.local_variables_initializer()
global_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run([local_op, global_op])
    out = sess.run(miou)